Thinking about grouping operations.
We have data contained in a pandas object.
- Split into groups based on one or more keys
  - Split is performed on the indicated axis(axis="index" or axis="columns")
- Function is applied to each group, producing a new value
- Results of all the function applications are combined into a result object.
The form of the resulting object will usually depend on what's being done to the data

Each gropuing key can take many forms. And the keys don´t all have of the same type
- list of arrays with same length as the axies being grouped
- value indicating a column name
- dictionary or series giving correspondence between two values

In [9]:
import numpy as np
import pandas as pd

In [10]:
df = pd.DataFrame({"key1":['a','a',None,'b','b','a',None],"key2":pd.Series([1,2,1,2,1,None,1],dtype="Int64"),"data1":np.random.standard_normal(7),"data2":np.random.standard_normal(7)})

In [11]:
df

,key1,key2,data1,data2
0,a,1,-2.537151,0.606848
1,a,2,-1.224228,0.360268
2,None,1,-0.385288,0.884701
3,b,2,-0.481568,0.022221
4,b,1,-1.288392,0.130018
5,a,<NA>,0.602720,0.350451
6,None,1,0.268225,0.297974


In [12]:
#Compute the mean of data1 based on key1 column
grouped_data1_key1 = df['data1'].groupby(df['key1'])
grouped_data1_key1 #This creates an groupby object which we still havent set how we want to get aggregated

In [13]:
grouped_data1_key1.sum()

key1
a   -3.158659
b   -1.769959
Name: data1, dtype: float64

In [14]:
grouped_data1_key1.mean()

key1
a   -1.052886
b   -0.884980
Name: data1, dtype: float64

In [15]:
grouped_data1_key_1_2 = df['data1'].groupby([df['key1'],df['key2']])

In [16]:
grouped_data1_key_1_2.mean() # The resulting data has hierarchical index

key1  key2
a     1      -2.537151
      2      -1.224228
b     1      -1.288392
      2      -0.481568
Name: data1, dtype: float64

In [17]:
grouped_data1_key_1_2.mean().unstack()

key2,1,2
key1,,
a,-2.537151,-1.224228
b,-1.288392,-0.481568


In [18]:
states = np.array(['OH','CA','CA','OH','OH','CA','OH'])
years = [2005,2005,2006,2006,2005,2006,2005]
#This two arrays could correspond to columns in the dataframe
df['data1'].groupby([states,years]).mean()

CA  2005   -1.224228
    2006    0.108716
OH  2005   -1.185772
    2006   -0.481568
Name: data1, dtype: float64

In [19]:
df.groupby("key1").mean() #The whole df will group by the column indicated, as long as column to group are numbers

,key2,data1,data2
key1,,,
a,1.5,-1.052886,0.439189
b,1.5,-0.884980,0.076120


In [20]:
keypp = df.groupby(df["key2"])

In [21]:
keypp.sum()

,key1,data1,data2
key2,,,
1,ab,-3.942606,1.919541
2,ab,-1.705795,0.382489


In [22]:
#Size
df.groupby(['key1','key2']).size()
#Any missings values in a group key are excluded. Can be disabled with dropna=False

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [23]:
df.groupby(['key1','key2'],dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [24]:
df.groupby('key1').count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


In [25]:
df

,key1,key2,data1,data2
0,a,1,-2.537151,0.606848
1,a,2,-1.224228,0.360268
2,None,1,-0.385288,0.884701
3,b,2,-0.481568,0.022221
4,b,1,-1.288392,0.130018
5,a,<NA>,0.602720,0.350451
6,None,1,0.268225,0.297974


## Iterating over groups
The object returned by groupby supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data

In [26]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1 -2.537151  0.606848
1    a     2 -1.224228  0.360268
5    a  <NA>  0.602720  0.350451
b
  key1  key2     data1     data2
3    b     2 -0.481568  0.022221
4    b     1 -1.288392  0.130018


In [27]:
# In case of multiple keys. The first element in the tuple will be another tuple of key values
for (k1,k2), group in df.groupby(['key1','key2']):
    print((k1, k2))
    print(group)

('a', 1)
  key1  key2     data1     data2
0    a     1 -2.537151  0.606848
('a', 2)
  key1  key2     data1     data2
1    a     2 -1.224228  0.360268
('b', 1)
  key1  key2     data1     data2
4    b     1 -1.288392  0.130018
('b', 2)
  key1  key2     data1     data2
3    b     2 -0.481568  0.022221


In [28]:
pieces = {name: group for name, group in df.groupby("key1")} #Generates a dictionary of key-> names, value-> group

In [29]:
pieces

{'a':   key1  key2     data1     data2
 0    a     1 -2.537151  0.606848
 1    a     2 -1.224228  0.360268
 5    a  <NA>  0.602720  0.350451,
 'b':   key1  key2     data1     data2
 3    b     2 -0.481568  0.022221
 4    b     1 -1.288392  0.130018}

In [30]:
pieces['b']

,key1,key2,data1,data2
3,b,2,-0.481568,0.022221
4,b,1,-1.288392,0.130018


In [31]:
#By default groupby groups on axis= index. We can change this way of grouping
# Here we group by the name of the columns
grouped = df.groupby({'key1':'key',"key2":"key","data1":'data','data2':"data"},axis="columns")

C:\Users\bruno\AppData\Local\Temp\ipykernel_11740\3987266796.py:3: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped = df.groupby({'key1':'key',"key2":"key","data1":'data','data2':"data"},axis="columns")


In [32]:
for group_key,group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0 -2.537151  0.606848
1 -1.224228  0.360268
2 -0.385288  0.884701
3 -0.481568  0.022221
4 -1.288392  0.130018
5  0.602720  0.350451
6  0.268225  0.297974
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


### Selecting a column or subset of columns
If we create a groupby using a column name or array of column names, has the effect of column subsetting


In [33]:
df.groupby('key2')[['data2']].sum()

,data2
key2,
1,1.919541
2,0.382489


In [34]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    1     0.606848
     2     0.360268
b    1     0.130018
     2     0.022221

### Grouping with dictionaries and Series
Grouping information may exists in a form other than an array.

In [35]:
people = pd.DataFrame(np.random.standard_normal((5,5)),columns=['a','b','c','d','e'],index=['Joe','Steve','Wanda','Jill','Trey'])
people.loc['Wanda',['b','c']]= np.nan
people

,a,b,c,d,e
Joe,-1.961162,-0.116541,0.581146,-1.267109,-0.313330
Steve,-1.136944,1.862932,0.169463,-0.417056,0.483774
Wanda,0.805999,NaN,NaN,-0.066514,0.368640
Jill,2.045071,-0.287334,0.528867,0.109960,1.704783
Trey,0.000877,0.852301,-0.545348,-0.106281,0.722926


In [36]:
#We could have a group correspondence for the columns and want to sum them
mapping = {'a':'red','b':'red','c':'blue','d':'blue','e':'red','f':'orange'}
# We could construct an array from the dictionary and pass to the groupby. We can just pass the dictionary
by_column = people.groupby(mapping,axis='columns')

C:\Users\bruno\AppData\Local\Temp\ipykernel_11740\80070685.py:4: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  by_column = people.groupby(mapping,axis='columns')


In [37]:
by_column.sum()

,blue,red
Joe,-0.685962,-2.391034
Steve,-0.247593,1.209762
Wanda,-0.066514,1.174639
Jill,0.638826,3.462520
Trey,-0.651629,1.576104


In [38]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [39]:
people.groupby(map_series,axis='columns').count()

C:\Users\bruno\AppData\Local\Temp\ipykernel_11740\1588593158.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  people.groupby(map_series,axis='columns').count()


,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### Grouping with functions
Using functions is a more generic way to group than dictionary or Series.
Any function passed as a group will be called once per index value. (Or column value if using axis='columns), with the return values being used as the group names

In [40]:
# using len function
people.groupby(len).sum() 

,a,b,c,d,e
3,-1.961162,-0.116541,0.581146,-1.267109,-0.313330
4,2.045948,0.564966,-0.016481,0.003678,2.427709
5,-0.330945,1.862932,0.169463,-0.483570,0.852414


In [41]:
people.groupby([len,['three','five','five','four','four']]).sum()


,,a,b,c,d,e
3,three,-1.961162,-0.116541,0.581146,-1.267109,-0.313330
4,four,2.045948,0.564966,-0.016481,0.003678,2.427709
5,five,-0.330945,1.862932,0.169463,-0.483570,0.852414


### Grouping by index levels
Hierarchically indexed datasets. Ability to aggregate using one of the levels of an axis index

In [42]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],[1,3,5,1,3,]],names=['cty','tenor'])

In [43]:
hier_df = pd.DataFrame(np.random.standard_normal((5,5)),columns=columns)

In [44]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.765233  0.216249 -1.022783  0.480313 -0.488139
1     -0.235217  0.636410  0.719210  0.699824 -0.697160
2     -0.559532 -1.119608 -0.209835 -1.683379 -0.307946
3      0.498098 -1.039348 -0.339780  0.267900  0.300958
4     -0.632997 -0.876689 -1.225322 -1.919040 -0.177687

In [45]:
hier_df.groupby(level='cty',axis='columns').count()

C:\Users\bruno\AppData\Local\Temp\ipykernel_11740\248999880.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  hier_df.groupby(level='cty',axis='columns').count()


cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3
4,2,3


### Data aggregation
Refers to any data transformation that produces scalar values from array
- Examples -> mean,count, sum, etc, any, all, cumsum, cumprod, first, last, median, min, max


In [46]:
df

,key1,key2,data1,data2
0,a,1,-2.537151,0.606848
1,a,2,-1.224228,0.360268
2,None,1,-0.385288,0.884701
3,b,2,-0.481568,0.022221
4,b,1,-1.288392,0.130018
5,a,<NA>,0.602720,0.350451
6,None,1,0.268225,0.297974


In [47]:
#nsmallest -> Selects the smallet requested number of values from the data (in order)
grouped = df.groupby('key1')
grouped['data2'].nsmallest(2) #It will return two values of each group

key1   
a     5    0.350451
      1    0.360268
b     3    0.022221
      4    0.130018
Name: data2, dtype: float64

In [48]:
#To use our own aggregation we can use agg
def own_aggregation(arr):
    for ar in arr:
        if(ar>0.5):
            return ar*1000
        else:
            return ar
    

grouped.agg(own_aggregation)

,key2,data1,data2
key1,,,
a,1000,-2.537151,606.848073
b,2000,-0.481568,0.022221


In [49]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0 -1.052886  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0 -0.884980  ...   

                         data2                                         \
           75%       max count      mean       std       min      25%   
key1                                                                    
a    -0.310754  0.602720   3.0  0.439189  0.145280  0.350451  0.35536   
b    -0.683274 -0.481568   2.0  0.076120  0.076224  0.022221  0.04917   

                                    
           50%       75%       max  
key1                                
a     0.360268  0.483558  0.606848  
b     0.076120  0.103069  0.130018  

[2 rows x 24 columns]

### Column-Wise and Multiple Function Application
We may want to aggregate using a function depending on the column, or using multiple functions at once

In [93]:
import random
dias=['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado','Domingo']
tipo = ['Almuerzo','Cena']
size= 90

cantidad = np.random.randint(20,80,size=size)
days= np.random.choice(dias,size=size,replace=True)
hue= np.random.choice(tipo,size=size,replace=True)
fuma = np.random.choice([True,False],size=size,replace=True)
total_bill = np.random.uniform(900,2000,size=size)
tip = total_bill/(np.random.randint(5,10,size=size))
df = pd.DataFrame({'dia':days,'hora':hue,'fuma':fuma,"cantidad":cantidad,'Total_bill':total_bill,"Tip":tip})



df

,dia,hora,fuma,cantidad,Total_bill,Tip
0,Lunes,Almuerzo,True,57,1827.155680,203.017298
1,Miercoles,Almuerzo,True,73,1685.628632,337.125726
2,Martes,Cena,False,68,1119.958184,159.994026
3,Domingo,Cena,False,75,1755.748284,292.624714
4,Martes,Cena,False,78,1720.952872,215.119109
...,...,...,...,...,...,...
85,Domingo,Almuerzo,True,58,1687.327604,210.915951
86,Martes,Almuerzo,True,23,1783.412224,297.235371
87,Viernes,Almuerzo,False,39,1603.006904,320.601381
88,Domingo,Cena,False,22,1668.788833,238.398405


In [97]:
df['tip_perc']= df['Tip']/df['Total_bill']
df_tips = df

In [52]:
df.drop(columns='hora')

,dia,fuma,cantidad,Total_bill,Tip,tip_perc
0,Jueves,True,43,1111.933372,222.386674,0.200000
1,Domingo,True,28,1565.305541,173.922838,0.111111
2,Domingo,False,78,1395.041681,232.506947,0.166667
3,Domingo,True,35,1134.144662,141.768083,0.125000
4,Lunes,False,59,1374.504793,274.900959,0.200000
...,...,...,...,...,...,...
85,Jueves,True,66,1156.538528,192.756421,0.166667
86,Sabado,True,33,1782.853978,198.094886,0.111111
87,Domingo,True,65,1947.942167,389.588433,0.200000
88,Domingo,False,26,1229.433984,245.886797,0.200000


In [53]:
#If you pass a list of functions, it will return the DataFrame with column names taken from the functions
grouped = df.groupby(['dia','fuma'])
grouped_pct = grouped['tip_perc']
grouped_pct.agg("mean")

dia        fuma 
Domingo    False    0.165646
           True     0.146737
Jueves     False    0.154138
           True     0.158045
Lunes      False    0.173571
           True     0.159921
Martes     False    0.170476
           True     0.137599
Miercoles  False    0.150397
           True     0.149735
Sabado     False    0.162143
           True     0.111111
Viernes    False    0.166667
           True     0.140829
Name: tip_perc, dtype: float64

In [54]:
grouped_pct.agg(['mean','std','sum',own_aggregation])

mean       std       sum  own_aggregation
dia       fuma                                                
Domingo   False  0.165646  0.035036  1.159524         0.166667
          True   0.146737  0.033995  1.320635         0.111111
Jueves    False  0.154138  0.030005  1.078968         0.166667
          True   0.158045  0.036132  1.738492         0.200000
Lunes     False  0.173571  0.036735  0.867857         0.200000
          True   0.159921  0.034607  0.959524         0.200000
Martes    False  0.170476  0.028651  0.852381         0.200000
          True   0.137599  0.028529  1.100794         0.142857
Miercoles False  0.150397  0.038000  1.052778         0.125000
          True   0.149735  0.035017  0.898413         0.166667
Sabado    False  0.162143  0.035319  0.810714         0.125000
          True   0.111111  0.000000  0.333333         0.111111
Viernes   False  0.166667  0.000000  0.333333         0.166667
          True   0.140829  0.028391  1.267460         0.142857

In [55]:
grouped_pct.agg([('Average','mean'),("Stdev",np.std),('My aggregation' , own_aggregation)])

C:\Users\bruno\AppData\Local\Temp\ipykernel_11740\1719398672.py:1: FutureWarning: The provided callable <function std at 0x000002693DFDF380> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  grouped_pct.agg([('Average','mean'),("Stdev",np.std),('My aggregation' , own_aggregation)])


Average     Stdev  My aggregation
dia       fuma                                     
Domingo   False  0.165646  0.035036        0.166667
          True   0.146737  0.033995        0.111111
Jueves    False  0.154138  0.030005        0.166667
          True   0.158045  0.036132        0.200000
Lunes     False  0.173571  0.036735        0.200000
          True   0.159921  0.034607        0.200000
Martes    False  0.170476  0.028651        0.200000
          True   0.137599  0.028529        0.142857
Miercoles False  0.150397  0.038000        0.125000
          True   0.149735  0.035017        0.166667
Sabado    False  0.162143  0.035319        0.125000
          True   0.111111  0.000000        0.111111
Viernes   False  0.166667  0.000000        0.166667
          True   0.140829  0.028391        0.142857

In [56]:
# We can apply the aggregations to multiple columns
functions=['mean','count',own_aggregation]
result= grouped[['tip_perc','Total_bill']].agg(functions)
result

tip_perc                         Total_bill        \
                     mean count own_aggregation         mean count   
dia       fuma                                                       
Domingo   False  0.165646     7        0.166667  1414.418198     7   
          True   0.146737     9        0.111111  1496.703747     9   
Jueves    False  0.154138     7        0.166667  1313.306513     7   
          True   0.158045    11        0.200000  1372.135046    11   
Lunes     False  0.173571     5        0.200000  1474.206761     5   
          True   0.159921     6        0.200000  1319.261659     6   
Martes    False  0.170476     5        0.200000  1590.150809     5   
          True   0.137599     8        0.142857  1470.637806     8   
Miercoles False  0.150397     7        0.125000  1432.117984     7   
          True   0.149735     6        0.166667  1577.089416     6   
Sabado    False  0.162143     5        0.125000  1482.245509     5   
          True   0.111111     3        0.111111  1618.277518     3   
Viernes   False  0.166667     2        0.166667  1406.134784     2   
          True   0.140829     9        0.142857  1285.409895     9   

                                 
                own_aggregation  
dia       fuma                   
Domingo   False    1.395042e+06  
          True     1.565306e+06  
Jueves    False    1.081614e+06  
          True     1.111933e+06  
Lunes     False    1.374505e+06  
          True     1.245955e+06  
Martes    False    1.993401e+06  
          True     9.735242e+05  
Miercoles False    1.246777e+06  
          True     1.617259e+06  
Sabado    False    1.745395e+06  
          True     1.690037e+06  
Viernes   False    1.032166e+06  
          True     1.354703e+06

We can also apply different functions to different columns

In [57]:
grouped.agg({"Tip":[np.max,'min','sum'],"cantidad":"sum"}) # With a dictionary we add a different aggregation for each column

C:\Users\bruno\AppData\Local\Temp\ipykernel_11740\2899162267.py:1: FutureWarning: The provided callable <function max at 0x000002693DFDE840> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  grouped.agg({"Tip":[np.max,'min','sum'],"cantidad":"sum"}) # With a dictionary we add a different aggregation for each column


Tip                          cantidad
                        max         min          sum      sum
dia       fuma                                               
Domingo   False  277.018563  171.824101  1583.120540      320
          True   389.588433  141.768083  2040.150337      352
Jueves    False  281.437696  169.709107  1384.419405      322
          True   373.481968  131.083778  2414.446565      603
Lunes     False  321.057497  170.658991  1274.112006      221
          True   284.980474  168.852266  1263.929556      295
Martes    False  398.680112  136.966314  1403.736590      252
          True   362.223578  120.302731  1635.734019      397
Miercoles False  329.345580  155.847181  1489.806794      338
          True   311.255678  157.302646  1399.970113      408
Sabado    False  356.181890  146.390119  1198.628327      233
          True   198.094886  153.549046   539.425839      101
Viernes   False  296.683924  172.027670   468.711595      109
          True   215.496067  156.377034  1595.688329      438

### Returning aggregated Data without Row Indexes
We don´t always want the information to be retrieved with an index.

In [58]:
df.drop(columns='hora')
grouped = df.groupby(['dia','fuma','hora'],as_index=False)

In [59]:
grouped.mean()

,dia,fuma,hora,cantidad,Total_bill,Tip,tip_perc
0,Domingo,False,Almuerzo,59.333333,1333.022824,216.739281,0.163889
1,Domingo,False,Cena,35.500000,1475.464729,233.225674,0.166964
2,Domingo,True,Almuerzo,31.500000,1340.848454,167.606057,0.125000
3,Domingo,True,Cena,41.285714,1541.233830,243.562603,0.152948
4,Jueves,False,Almuerzo,30.000000,1347.608038,179.779056,0.138889
5,Jueves,False,Cena,52.400000,1299.585903,204.972259,0.160238
6,Jueves,True,Almuerzo,41.500000,1747.110491,302.941780,0.171429
7,Jueves,True,Cena,57.777778,1288.807169,200.951445,0.155071
8,Lunes,False,Almuerzo,35.000000,1448.354735,255.539149,0.175000
9,Lunes,False,Cena,58.000000,1512.984799,253.747280,0.171429


Apply: General split-apply-combine
The most general groupby method is apply.
Apply:
- Splits the object being manipulated into pieces
- Invokes the passed function on each piece
- Attempts to concatenate the pieces

In [60]:
def top(df, n=2,column="tip_perc"):
    return df.sort_values(column,ascending=False)[:n]
top(df,n=2,column="Total_bill")




,dia,hora,fuma,cantidad,Total_bill,Tip,tip_perc
10,Martes,Almuerzo,False,51,1993.400559,398.680112,0.200000
27,Miercoles,Almuerzo,True,74,1992.345947,221.371772,0.111111


In [61]:
# We can group this by smoker, and call apply
df.groupby('dia').apply(top)
# First, splits into groups based on the value of smoker
# Second, top function is called on each group
# The results of each function are glued togethe using pandas.concate, labelling the pieces with the group names

C:\Users\bruno\AppData\Local\Temp\ipykernel_11740\1725809158.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('dia').apply(top)


dia      hora   fuma  cantidad   Total_bill         Tip  \
dia                                                                           
Domingo   82    Domingo      Cena   True        20  1795.711215  359.142243   
          87    Domingo      Cena   True        65  1947.942167  389.588433   
Jueves    89     Jueves      Cena   True        67  1241.215831  248.243166   
          29     Jueves  Almuerzo   True        26  1867.409840  373.481968   
Lunes     4       Lunes  Almuerzo  False        59  1374.504793  274.900959   
          66      Lunes  Almuerzo   True        30  1424.902372  284.980474   
Martes    10     Martes  Almuerzo  False        51  1993.400559  398.680112   
          19     Martes      Cena  False        45  1880.772437  376.154487   
Miercoles 15  Miercoles      Cena  False        31  1646.727899  329.345580   
          11  Miercoles      Cena  False        32  1270.568234  254.113647   
Sabado    78     Sabado      Cena  False        36  1212.453223  242.490645   
          32     Sabado  Almuerzo  False        45  1780.909451  356.181890   
Viernes   23    Viernes      Cena   True        21  1077.480334  215.496067   
          13    Viernes      Cena  False        48  1032.166022  172.027670   

              tip_perc  
dia                     
Domingo   82  0.200000  
          87  0.200000  
Jueves    89  0.200000  
          29  0.200000  
Lunes     4   0.200000  
          66  0.200000  
Martes    10  0.200000  
          19  0.200000  
Miercoles 15  0.200000  
          11  0.200000  
Sabado    78  0.200000  
          32  0.200000  
Viernes   23  0.200000  
          13  0.166667

In [62]:
result = df.groupby('fuma')['tip_perc'].describe()
result

,count,mean,std,min,25%,50%,75%,max
fuma,,,,,,,,
False,38.0,0.161988,0.031896,0.111111,0.129464,0.166667,0.200000,0.2
True,52.0,0.146513,0.032496,0.111111,0.125000,0.142857,0.166667,0.2


In [63]:
result.unstack('fuma')

       fuma 
count  False    38.000000
       True     52.000000
mean   False     0.161988
       True      0.146513
std    False     0.031896
       True      0.032496
min    False     0.111111
       True      0.111111
25%    False     0.129464
       True      0.125000
50%    False     0.166667
       True      0.142857
75%    False     0.200000
       True      0.166667
max    False     0.200000
       True      0.200000
dtype: float64

### Supressing the group keys
We can disabled the group  by using group_keys=False 


In [64]:
df.groupby('fuma',group_keys=False).apply(top)

C:\Users\bruno\AppData\Local\Temp\ipykernel_11740\2175587835.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('fuma',group_keys=False).apply(top)


,dia,hora,fuma,cantidad,Total_bill,Tip,tip_perc
62,Jueves,Cena,False,25,1407.188481,281.437696,0.2
4,Lunes,Almuerzo,False,59,1374.504793,274.900959,0.2
89,Jueves,Cena,True,67,1241.215831,248.243166,0.2
23,Viernes,Cena,True,21,1077.480334,215.496067,0.2


### Quantiles
We can combine pandas.cut and pandas.qcut -> Slicing data up into buckets with bins of your choosing, or by sample quantiles.


In [65]:
frame = pd.DataFrame({'data1':np.random.standard_normal(1000),'data2':np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,1.221064,0.661044
1,-0.132013,1.425624
2,-0.179109,-0.260086
3,1.053936,-0.263201
4,2.328813,-0.121079


In [66]:
quartiles = pd.cut(frame['data1'],4)
quartiles.head()

0     (1.197, 2.708]
1    (-0.314, 1.197]
2    (-0.314, 1.197]
3    (-0.314, 1.197]
4     (1.197, 2.708]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.343, -1.826] < (-1.826, -0.314] < (-0.314, 1.197] < (1.197, 2.708]]

The categorical object returned by cut can be passed directly to groupby.
So we can compute a set of group statistics for the quartiles

In [67]:
def get_stats(group):
    return pd.DataFrame(
        {'min':group.min(),'max':group.max(),'mean':group.mean(),'count':group.count()}
    )
grouped = frame.groupby(quartiles)
grouped.apply(get_stats)

C:\Users\bruno\AppData\Local\Temp\ipykernel_11740\3749917180.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = frame.groupby(quartiles)


min       max      mean  count
data1                                                      
(-3.343, -1.826] data1 -3.336965 -1.834749 -2.204582     24
                 data2 -1.770725  1.874640  0.063387     24
(-1.826, -0.314] data1 -1.818696 -0.314436 -0.933371    367
                 data2 -3.332119  2.695315 -0.014452    367
(-0.314, 1.197]  data1 -0.313538  1.194515  0.346894    485
                 data2 -2.805328  2.952470 -0.019708    485
(1.197, 2.708]   data1  1.202966  2.708417  1.612822    124
                 data2 -2.301180  2.925191  0.030076    124

In [68]:
quartiles_samp = pd.qcut(frame['data1'],4,labels=False)
quartiles_samp.head()

0    3
1    1
2    1
3    3
4    3
Name: data1, dtype: int64

In [69]:
grouped = frame.groupby(quartiles_samp).apply(get_stats)
grouped

min       max      mean  count
data1                                           
0     data1 -3.336965 -0.715414 -1.292914    250
      data2 -3.332119  2.681338  0.050453    250
1     data1 -0.710256 -0.047067 -0.367141    250
      data2 -2.618781  2.695315 -0.089773    250
2     data1 -0.044735  0.685368  0.292552    250
      data2 -2.805328  2.952470 -0.074115    250
3     data1  0.690609  2.708417  1.258607    250
      data2 -2.301180  2.925191  0.074989    250

### Example: Filling missing data with group-specific values
We can use fillna with groupby to fill the missing values

In [70]:
s= pd.Series(np.random.standard_normal(6))
s

0    1.420364
1   -0.105536
2    0.656054
3    0.374714
4    0.638677
5   -0.860476
dtype: float64

In [71]:
s[::2] = np.nan
s

0         NaN
1   -0.105536
2         NaN
3    0.374714
4         NaN
5   -0.860476
dtype: float64

In [72]:
s.fillna(s.mean())

0   -0.197099
1   -0.105536
2   -0.197099
3    0.374714
4   -0.197099
5   -0.860476
dtype: float64

In [73]:
# Let's suppose we neewd to fill value to vary by group
states = ['Ohio','Nueva York', 'Vermont','Florida','Oregon','Nevada','California','Idaho']
group_key = ['East','East','East','East','West','West','West','West']
data = pd.Series(np.random.standard_normal(8),index=states)
data

Ohio          0.563322
Nueva York   -0.700172
Vermont      -0.878307
Florida       0.753726
Oregon        0.548660
Nevada       -0.382561
California   -0.120247
Idaho         1.224136
dtype: float64

In [74]:
data[['Vermont','Florida','Nevada']]=np.nan

In [75]:
data

Ohio          0.563322
Nueva York   -0.700172
Vermont            NaN
Florida            NaN
Oregon        0.548660
Nevada             NaN
California   -0.120247
Idaho         1.224136
dtype: float64

In [76]:
data.groupby(group_key).size()

East    4
West    4
dtype: int64

In [77]:
data.groupby(group_key).count() #It doesn't count nan values

East    2
West    3
dtype: int64

In [78]:
data.groupby(group_key).mean()

East   -0.068425
West    0.550850
dtype: float64

In [79]:
fill_values= {'East':0.4,'West':1.9}
def fill_func(group):
    return group.fillna(fill_values[group.name])

In [80]:
data.groupby(group_key).apply(fill_func)

East  Ohio          0.563322
      Nueva York   -0.700172
      Vermont       0.400000
      Florida       0.400000
West  Oregon        0.548660
      Nevada        1.900000
      California   -0.120247
      Idaho         1.224136
dtype: float64

### Random Sampling and Permutation
Want to dra a random sample from a large dataset.
There are different ways to do it. Here we use "Sample Method for series"


In [84]:
palos = ['Espada','Oro','Basto','Copa']
palos = ['E','O','B','C']
valor_carta = list(range(1,13))

cartas =[]

for palo in palos:
    cartas.extend(str(num) + palo for num in valor_carta)
deck = pd.Series(48,index=cartas)
deck.head()

1E    48
2E    48
3E    48
4E    48
5E    48
dtype: int64

In [87]:
def draw(deck,n=5):
    return deck.sample(n)
draw(deck)


7B    48
1O    48
2E    48
1C    48
5C    48
dtype: int64

In [88]:
df = pd.DataFrame({"category":['a','a','a','a','b','b','b','b'],'data':np.random.standard_normal(8),'weights':np.random.standard_normal(8)})
df

,category,data,weights
0,a,0.933912,0.743611
1,a,-0.047716,-0.837593
2,a,-0.232751,0.301079
3,a,-1.551671,-1.590184
4,b,0.615702,0.079690
5,b,2.298146,-1.596808
6,b,-0.850020,0.322434
7,b,-0.814958,0.291714


In [91]:
grouped = df.groupby("category")
def get_wavg(group):
    return np.average(group['data'],weights=group['weights'])



C:\Users\bruno\AppData\Local\Temp\ipykernel_11740\366174538.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped.apply(get_wavg)


category
a   -2.264355
b    4.576501
dtype: float64

In [92]:
grouped.apply(get_wavg)

C:\Users\bruno\AppData\Local\Temp\ipykernel_11740\4286695940.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped.apply(get_wavg)


category
a   -2.264355
b    4.576501
dtype: float64

### Pivot table and Cross-Tabulation
Pivot table -> Is a data summatization tool, frequently found in a spreadsheet programs and data analysis software.
It aggregates a table of data by one or more keys, arranging the data in a rectangle with some of the group keys along the rows and some along the columns.
In pandas, pivot table are possible through groupby, combined with reshape operations utilizing hierarchical indexing.
In pandas, there algo is pivot_table method and pandas.pivot_table.
Also pivot_table can add partial totals, also known as margins.

In [98]:
df_tips.head()

,dia,hora,fuma,cantidad,Total_bill,Tip,tip_perc
0,Lunes,Almuerzo,True,57,1827.155680,203.017298,0.111111
1,Miercoles,Almuerzo,True,73,1685.628632,337.125726,0.200000
2,Martes,Cena,False,68,1119.958184,159.994026,0.142857
3,Domingo,Cena,False,75,1755.748284,292.624714,0.166667
4,Martes,Cena,False,78,1720.952872,215.119109,0.125000


In [100]:
df_tips.pivot_table(index=['dia','fuma','hora'])

Tip   Total_bill   cantidad  tip_perc
dia       fuma  hora                                                  
Domingo   False Almuerzo  323.990095  1943.940572  71.000000  0.166667
                Cena      250.356727  1531.974190  40.666667  0.162897
          True  Almuerzo  237.989214  1570.107988  48.500000  0.152679
                Cena      192.696703  1359.504263  46.500000  0.145833
Jueves    False Almuerzo  221.802256  1652.919394  58.200000  0.138016
                Cena      249.259617  1609.763860  40.500000  0.154762
          True  Almuerzo  249.724500  1602.430464  39.571429  0.157710
                Cena      271.951882  1441.370021  53.500000  0.191667
Lunes     False Almuerzo  224.400496  1376.474675  47.500000  0.163095
                Cena      214.289605  1343.270893  56.666667  0.158730
          True  Almuerzo  192.238603  1367.227612  48.500000  0.155556
                Cena      274.945574  1758.233522  52.000000  0.154762
Martes    False Almuerzo  148.793385  1171.949234  47.333333  0.126323
                Cena      207.275835  1479.342742  59.000000  0.145106
          True  Almuerzo  233.354752  1567.583393  48.750000  0.155159
                Cena      272.524377  1635.146259  59.000000  0.166667
Miercoles False Cena      181.265533  1558.673674  38.333333  0.115741
          True  Almuerzo  271.004912  1686.899103  49.000000  0.159921
                Cena      103.148771   928.338942  46.000000  0.111111
Sabado    False Almuerzo  217.032951  1519.230655  52.000000  0.142857
                Cena      199.335535  1616.869886  36.666667  0.126323
          True  Cena      161.052884  1240.792713  49.400000  0.128571
Viernes   False Almuerzo  230.877659  1426.088763  56.166667  0.160582
                Cena      159.501058  1267.330148  62.500000  0.126984
          True  Almuerzo  208.237023  1459.119461  60.000000  0.145370
                Cena      233.622653  1357.564973  56.000000  0.162500

In [103]:
df_tips.pivot_table(index=['hora','dia'],columns='fuma',values=['cantidad','tip_perc'],margins=True)

cantidad                        tip_perc            \
fuma                    False       True        All     False      True   
hora     dia                                                              
Almuerzo Domingo    71.000000  48.500000  53.000000  0.166667  0.152679   
         Jueves     58.200000  39.571429  47.333333  0.138016  0.157710   
         Lunes      47.500000  48.500000  47.833333  0.163095  0.155556   
         Martes     47.333333  48.750000  48.142857  0.126323  0.155159   
         Miercoles        NaN  49.000000  49.000000       NaN  0.159921   
         Sabado     52.000000        NaN  52.000000  0.142857       NaN   
         Viernes    56.166667  60.000000  57.444444  0.160582  0.145370   
Cena     Domingo    40.666667  46.500000  42.125000  0.162897  0.145833   
         Jueves     40.500000  53.500000  47.000000  0.154762  0.191667   
         Lunes      56.666667  52.000000  54.800000  0.158730  0.154762   
         Martes     59.000000  59.000000  59.000000  0.145106  0.166667   
         Miercoles  38.333333  46.000000  40.250000  0.115741  0.111111   
         Sabado     36.666667  49.400000  44.625000  0.126323  0.128571   
         Viernes    62.500000  56.000000  59.250000  0.126984  0.162500   
All                 50.276596  49.116279  49.722222  0.147188  0.154780   

                              
fuma                     All  
hora     dia                  
Almuerzo Domingo    0.155476  
         Jueves     0.149504  
         Lunes      0.160582  
         Martes     0.142800  
         Miercoles  0.159921  
         Sabado     0.142857  
         Viernes    0.155511  
Cena     Domingo    0.158631  
         Jueves     0.173214  
         Lunes      0.157143  
         Martes     0.148186  
         Miercoles  0.114583  
         Sabado     0.127728  
         Viernes    0.144742  
All                 0.150816

In [106]:
df_tips.pivot_table(index=['hora','fuma'],columns='dia',values='tip_perc',aggfunc=len,margins=True,fill_value=100,margins_name='Totales')

dia             Domingo  Jueves  Lunes  Martes  Miercoles  Sabado  Viernes  \
hora     fuma                                                                
Almuerzo False        1       5      4       3        100       1        6   
         True         4       7      2       4          6     100        3   
Cena     False        6       4      3       6          3       3        2   
         True         2       4      2       1          1       5        2   
Totales              13      20     11      14         10       9       13   

dia             Totales  
hora     fuma            
Almuerzo False       20  
         True        26  
Cena     False       27  
         True        17  
Totales              90

### Cross-Tabulations: Crosstab
Is a special case of a pivot table that computes group frequencies.


In [109]:
pd.crosstab([df_tips['hora'],df_tips['fuma']],df_tips['dia'],margins=True)

dia             Domingo  Jueves  Lunes  Martes  Miercoles  Sabado  Viernes  \
hora     fuma                                                                
Almuerzo False        1       5      4       3          0       1        6   
         True         4       7      2       4          6       0        3   
Cena     False        6       4      3       6          3       3        2   
         True         2       4      2       1          1       5        2   
All                  13      20     11      14         10       9       13   

dia             All  
hora     fuma        
Almuerzo False   20  
         True    26  
Cena     False   27  
         True    17  
All              90